In [1]:
import pandas as pd

In [51]:
# Updating parking zone input file for future years (parking_costs.csv)
df = pd.read_csv('R:\e2projects_two\SoundCast\Inputs\db_inputs\parking_costs.csv')

# Develop data for 2035 by interpolating between 2030 and 2040
df_2030 = df[df['year'] == 2030].copy()
df_2040 = df[df['year'] == 2040].copy()
df_2035 = df_2030.merge(df_2040, on='ENS', suffixes=('_2030', '_2040'))
df_2035['HR_COST'] = df_2035['HR_COST_2030'] + (df_2035['HR_COST_2040'] - df_2035['HR_COST_2030']) / 2
df_2035['DAY_COST'] = df_2035['DAY_COST_2030'] + (df_2035['DAY_COST_2040'] - df_2035['DAY_COST_2030']) / 2
df_2035[['HR_COST','DAY_COST']] = df_2035[['HR_COST','DAY_COST']].astype('int')
df_2035['year'] = 2035
df_2035 = df_2035[['ENS','HR_COST', 'DAY_COST', 'year']]
df = pd.concat([df,df_2035], ignore_index=True, sort=False, axis=0).fillna(0)

# Sort values by ensemble number
df['zone_id'] = df['ENS'].astype(str).apply(lambda x: x.split('gz')[-1]).astype('int')
df.sort_values(['year','zone_id'], inplace=True)
df.drop('zone_id', axis=1, inplace=True)
df.to_clipboard()

df.to_csv('R:\e2projects_two\SoundCast\Inputs\db_inputs\parking_costs.csv', index=False)

In [49]:
# median parking costs per year
df[df['HR_COST'] > 0].groupby('year').agg({'HR_COST':'median','DAY_COST':'median'}).reset_index()

,year,HR_COST,DAY_COST
0,2014,332,1464
1,2018,332,1464
2,2023,332,1464
3,2025,381,1119
4,2030,381,1161
5,2035,381,1161
6,2040,397,1161
7,2044,397,1161
8,2050,435,1161


In [71]:
# Analysis: compare parcel median costs to the ensemble input data


# Parking costs are available on parcels (via parking survey data, the latest being from 2022)

# We can compare the ensemble (zone) data for 2023 to see if it's reasonably matching the parcel data

# Base year data doesn't need to be udpated because parcel parking data is updated through urbansim
# 2023 was copied from 2018 but these data won't actually be used 
df_parcel = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\rtp_2050_updated_income\parcels_urbansim.txt', sep='\s+')

In [72]:
df_parcel.head()

,aparks,empedu_p,empfoo_p,empgov_p,empind_p,empmed_p,empofc_p,empoth_p,empret_p,emprsc_p,...,ppricdyp,pprichrp,sfunits,sqft_p,stugrd_p,stuhgh_p,stuuni_p,taz_p,xcoord_p,ycoord_p
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,60440,0,0,0,1019,1.292251e+06,162729.949912
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,5446,0,0,0,1018,1.291828e+06,164043.167618
2,0,0,2,0,0,0,0,0,2,0,...,0,0,1,3438,0,0,0,1018,1.291590e+06,164050.082322
3,0,0,0,0,4,3,18,8,5,0,...,0,0,0,5112,0,0,0,1018,1.291535e+06,164051.591361
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,6068,0,0,0,1018,1.291475e+06,164043.810513


In [73]:
# Load parking zone file
df_parking_zones = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\parking_zones.csv')
df_parking_costs = pd.read_csv(r'R:\e2projects_two\SoundCast\Inputs\db_inputs\parking_costs.csv')

In [74]:
df_parking = df_parking_costs.merge(df_parking_zones, on='ENS')
df_parking = df_parking[df_parking['year'] == 2050]

In [75]:
# df_parking

In [76]:
df_parcel_merge = df_parcel.merge(df_parking, left_on='taz_p', right_on='TAZ')

In [77]:
# Get median price for paid parking zones in each ensemble zone
df_results = df_results = df_parcel_merge[df_parcel_merge['ppricdyp'] > 0].groupby('ENS').median()[['HR_COST','DAY_COST','ppricdyp','pprichrp']]

In [78]:
df_results

,HR_COST,DAY_COST,ppricdyp,pprichrp
ENS,,,,
gz1,682.0,2561.0,1800.0,450.0
gz10,703.0,3066.0,2300.0,600.0
gz11,762.0,3283.0,1800.0,400.0
gz16,834.0,1359.0,1500.0,400.0
gz19,771.0,1414.0,800.0,200.0
gz2,254.0,1129.0,600.0,300.0
gz3,238.0,1491.0,1000.0,200.0
gz4,410.0,1682.0,1000.0,200.0
gz5,451.0,2043.0,1500.0,300.0


In [79]:
df_results = df_parcel_merge[df_parcel_merge['ppricdyp'] > 0].groupby('ENS').mean()[['HR_COST','DAY_COST','ppricdyp','pprichrp']]

In [80]:
df_results

,HR_COST,DAY_COST,ppricdyp,pprichrp
ENS,,,,
gz1,682.0,2561.0,1996.153846,446.153846
gz10,703.0,3066.0,2447.887324,521.126761
gz11,762.0,3283.0,2217.391304,404.347826
gz16,834.0,1359.0,1861.538462,376.923077
gz19,771.0,1414.0,800.000000,200.000000
gz2,254.0,1129.0,720.512821,310.256410
gz3,238.0,1491.0,935.294118,200.000000
gz4,410.0,1682.0,1025.531915,253.191489
gz5,451.0,2043.0,1411.904762,273.809524


In [81]:
# df_parking_costs[df_parking_costs['HR_COST'] > 0]['ENS'].unique()